In [1]:
!pip install dash
!pip install arch
!pip install dash-bootstrap-components

import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from arch import arch_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 11.4 MB/s eta 0:00:00


In [26]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

# App layout desgin
app.layout = dbc.Container([
    html.H1("Stock Price Prediction Dashboard", className="mb-4 text-center"),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Input Parameters", className="card-title"),
                    dbc.Label("Stock Ticker", html_for="ticker-input"), # Stock Ticker input
                    dbc.Input(id="ticker-input", type="text", value="", placeholder="e.g. AAPL"),

                    dbc.Label("Start Date", html_for="start-date"), # Start Date input
                    dcc.DatePickerSingle(
                        id='start-date',
                        min_date_allowed=datetime(2010, 1, 1),
                        max_date_allowed=datetime.today(),
                        initial_visible_month=datetime.today(),
                        placeholder="Select a start date"
                    ),

                    dbc.Label("End Date", html_for="end-date"), # End Date input
                    dcc.DatePickerSingle(
                        id='end-date',
                        min_date_allowed=datetime(2010, 1, 1),
                        max_date_allowed=datetime.today(),
                        initial_visible_month=datetime.today(),
                        placeholder="Select an end date"
                    ),

                    dbc.Label("Prediction Days", html_for="prediction-days"), # Prediction Days input
                    dcc.Dropdown(
                        id='prediction-days',
                        options=[
                            {'label': '5 days', 'value': 5},
                            {'label': '10 days', 'value': 10},
                            {'label': '30 days', 'value': 30}
                        ],
                        value=5,
                        clearable=False
                    ),

                    html.Br(),
                    dbc.Button("Run Analysis", id="run-button", color="primary", className="w-100") # Start Forecasting and Analysis
                ])
            ])
        ], md=4),

        # Here we design 5 tabs in total
        dbc.Col([
            dbc.Tabs([
                dbc.Tab(label="Historical Price", tab_id="tab-historical"),
                dbc.Tab(label="Monte Carlo Simulation", tab_id="tab-montecarlo"),
                dbc.Tab(label="Price Distribution", tab_id="tab-distribution"),
                dbc.Tab(label="Random Forest Prediction", tab_id="tab-randomforest"),
                dbc.Tab(label="Diversification Advisor", tab_id="tab-advisor"),
            ], id="tabs", active_tab="tab-historical"),

            html.Div(id="tab-content", className="p-4")
        ], md=8)
    ])
], fluid=True)

# Designing Fuctions for Historical Price, Monte Carlo Simulation, Price Distribution and Random Forest Prediction
#1. Fetch the choosen stock data from yfinance
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    if stock_data.empty:
        raise ValueError(f"No data found for ticker {ticker}")

    filename = f"{ticker}_{start_date}_{end_date}.csv"

    # Create headers of data
    header = pd.MultiIndex.from_tuples([
        ('Close', ticker),
        ('High', ticker),
        ('Low', ticker),
        ('Open', ticker),
        ('Volume', ticker),
    ])

    # Send data to CSV for review
    stock_data.columns = header
    stock_data.to_csv(filename)

    stock_data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

    # Calculate the log return to create a new column
    stock_data["Log_Return"] = np.log(stock_data["Close"] / stock_data["Close"].shift(1))
    stock_data.dropna(inplace=True)
    return stock_data

#2. Load the data from the downloaded CSV
def load_stock_data(ticker, start_date, end_date):
    filename = f"{ticker}_{start_date}_{end_date}.csv"
    try:
        stock_data = pd.read_csv(filename, header=[0,1], index_col=0, parse_dates=True)
        stock_data.columns = [col[0] if isinstance(col, tuple) else col for col in stock_data.columns]

        stock_data["Log_Return"] = np.log(stock_data["Close"] / stock_data["Close"].shift(1))
        stock_data.dropna(inplace=True)
        return stock_data
    except FileNotFoundError:
        return fetch_stock_data(ticker, start_date, end_date)

#3. Add the calculation of 50-day and 200-day average for market state
def determine_market_state(stock_data):
    stock_data["SMA_50"] = stock_data["Close"].rolling(window=50).mean()
    stock_data["SMA_200"] = stock_data["Close"].rolling(window=200).mean()
    stock_data["Market_State"] = np.where(stock_data["SMA_50"] > stock_data["SMA_200"], "Bull", "Bear")
    return stock_data

#4. Calculate historical average return
def calculate_historical_return(stock_data):
    return stock_data["Log_Return"].mean()

#5. Calculate volatility using GARCH(1,1) for Monte Carlo
def calculate_garch_volatility(stock_data):
    model = arch_model(stock_data["Log_Return"], vol="Garch", p=1, q=1)
    garch_fit = model.fit(disp="off")
    last_volatility = garch_fit.conditional_volatility.iloc[-1]
    return last_volatility

#6. Executed Monte Carlo Simulation for 1000 times
def monte_carlo_simulation(stock_data, days, num_simulations=1000):
    last_price = stock_data["Close"].iloc[-1]
    mean_return = calculate_historical_return(stock_data)
    stock_volatility = calculate_garch_volatility(stock_data)

    # Add the penalization system to future stock price based on market state since market sentiment will affect future stock prices
    if stock_data["Market_State"].iloc[-1] == "Bull":
        adjusted_return = mean_return * 1.2 # If Bull, multiple mean_return by 1.2
    else:
        adjusted_return = mean_return * 0.8 # If Bear, multiple mean_return by 0.8

    simulations = np.zeros((days + 1, num_simulations))
    simulations[0, :] = last_price

    # Get the path of each simulation
    for i in range(num_simulations):
        price_path = [last_price]
        for _ in range(days):
            shock = np.random.normal(adjusted_return, stock_volatility)
            price_path.append(price_path[-1] * np.exp(shock))

        simulations[:, i] = np.array(price_path).flatten()

    return simulations

#7. Executed Random Forests
def random_forest_prediction(stock_data, days):
    df = stock_data.copy()
    df["Target"] = df["Close"].shift(-days)

    # Use lag_days 10
    lag_days = 10
    for i in range(1, lag_days + 1):
        df[f"Close_Lag_{i}"] = df["Close"].shift(i)

    df.dropna(inplace=True)

    # Choose features and target
    features = [f"Close_Lag_{i}" for i in range(1, lag_days + 1)]
    X = df[features]
    y = df["Target"]

    # Split time series data into train and test set
    split_index = int(len(df) * 0.8)
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

    # Train random forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error: {mae:.2f}")

    # Forecast future stock prices
    latest_data = df[features].iloc[-1:].values
    future_prices = []

    for _ in range(days):
        pred_price = model.predict(latest_data)[0]
        future_prices.append(pred_price)
        latest_data = np.roll(latest_data, -1)
        latest_data[0, -1] = pred_price

    return [round(float(price), 2) for price in future_prices]

# Advisor tools:
# 1. Price trend classification based on prediction
def get_price_trend(current_price, predicted_price, threshold=0.01):
    pct_change = (predicted_price - current_price) / current_price
    if pct_change > threshold:
        return "up"
    elif pct_change < -threshold:
        return "down"
    else:
        return "stable"

# 2. Stock concentration risk analysis
def check_concentration_ratio(stock_value, total_portfolio_value, threshold=0.6):
    ratio = stock_value / total_portfolio_value
    if ratio > threshold:
        return ratio, "Stock concentration exceeds {:.0%}. Diversification is recommended.".format(threshold)
    else:
        return ratio, "Stock concentration is within acceptable range."

# 3. Simulate portfolio volatility after partial diversification (Modern Portfolio Theory)
def calculate_portfolio_volatility(std1, std2, w1, corr=0.2):
    w2 = 1 - w1
    portfolio_std = np.sqrt(
        (w1 * std1) ** 2 +
        (w2 * std2) ** 2 +
        2 * w1 * w2 * std1 * std2 * corr
    )
    return portfolio_std

# 4. Generate investment advice based on trend, concentration, and simulated volatility
def generate_advice(price_trend, concentration_ratio, current_vol, new_vol, threshold=0.6):
    advice = []

    # Trend logic
    if price_trend == "down":
        advice.append("The model predicts a short-term decline. Please be cautious.")
    elif price_trend == "up":
        advice.append("The model predicts a short-term increase. Consider holding.")
    else:
        advice.append("The model predicts a stable price trend.")

    # Concentration logic + diversification simulation
    if concentration_ratio > threshold:
        advice.append("High stock concentration detected. Consider diversifying.")
        if new_vol < current_vol:
            advice.append("Diversification reduces overall portfolio risk.")
            advice.append("Recommendation: Sell 20% and reinvest in low-volatility assets like S&P 500 ETFs.")
        else:
            advice.append("Diversification impact on risk is minimal. Decide based on personal goals.")
    else:
        advice.append("Stock concentration is healthy. No changes needed.")

    return "\n".join(advice)

# 5. Get benchmark volatility (SPY)
def get_benchmark_volatility():
    spy = yf.download("SPY", period="3y")
    spy["Log_Return"] = np.log(spy["Close"] / spy["Close"].shift(1))
    spy.dropna(inplace=True)
    return float(calculate_garch_volatility(spy))

# 6. Diversification advisor
def run_diversification_advisor(current_price, predicted_price, stock_vol, benchmark_volatility, stock_val, total_val, new_weight):
    trend = get_price_trend(current_price, predicted_price)
    ratio, _ = check_concentration_ratio(stock_val, total_val)
    new_vol = calculate_portfolio_volatility(stock_vol, benchmark_volatility, new_weight)
    return generate_advice(trend, ratio, stock_vol, new_vol)




# Callbacks and dashboard design
@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab"),
    Input("run-button", "n_clicks"),
    State("ticker-input", "value"),
    State("start-date", "date"),
    State("end-date", "date"),
    State("prediction-days", "value")
)
def update_tab_content(active_tab, n_clicks, ticker, start_date, end_date, prediction_days):
    if n_clicks is None:
        return dbc.Alert("Please enter parameters and click 'Run Analysis'", color="info")

    try:
        # Convert dates from string to datetime
        start_date = datetime.strptime(start_date[:10], "%Y-%m-%d")
        end_date = datetime.strptime(end_date[:10], "%Y-%m-%d")

        # Fetch and process data
        stock_data = load_stock_data(ticker, start_date, end_date)
        stock_data = determine_market_state(stock_data)
        stock_volatility = calculate_garch_volatility(stock_data)
        simulations = monte_carlo_simulation(stock_data, prediction_days)
        rf_predictions = random_forest_prediction(stock_data, prediction_days)

        # Diversification Advisor Calculation
        current_price = float(stock_data["Close"].iloc[-1])
        predicted_price = float(np.mean(simulations[-1]))
        stock_value = 50000
        total_portfolio_value = 80000
        benchmark_volatility = get_benchmark_volatility()


        new_weight = 0.8


        # Calculate statistics on confidence intervals, mean and median
        final_prices = simulations[-1]
        if isinstance(final_prices, pd.Series):
            final_prices = final_prices.values

        future_price_90CI = (round(float(np.percentile(final_prices, 5)), 2),
                            round(float(np.percentile(final_prices, 95)), 2))
        future_price_70CI = (round(float(np.percentile(final_prices, 15)), 2),
                            round(float(np.percentile(final_prices, 85)), 2))
        mean_price = round(float(np.mean(final_prices)), 2)
        median_price = round(float(np.median(final_prices)), 2)


        # Create content based on active tab
        # Historical Price Tab
        if active_tab == "tab-historical":
            # Ensure data is over 200 for moving averages
            if len(stock_data) < 200:
                raise ValueError("Not enough data points to calculate moving averages. Please select a longer time period.")

            stock_data.index = pd.to_datetime(stock_data.index)
            stock_data.fillna(method='ffill', inplace=True)
            fig = go.Figure()

            # Plot Lines for closing price, SMA50 and SMA200
            # Closing Price
            fig.add_trace(go.Scatter(
                x=stock_data.index,
                y=stock_data["Close"],
                mode='lines',
                name='Close Price',
                line=dict(color='#3366CC', width=2),
                hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Price: %{y:.2f}<extra></extra>'
            ))

            # SMA50
            fig.add_trace(go.Scatter(
                x=stock_data.index,
                y=stock_data["SMA_50"],
                mode='lines',
                name='50-day SMA',
                line=dict(color='#FF9933', width=1.5, dash='dot'),
                hovertemplate='<b>%{x|%Y-%m-%d}</b><br>SMA50: %{y:.2f}<extra></extra>'
            ))

            # SMA200
            fig.add_trace(go.Scatter(
                x=stock_data.index,
                y=stock_data["SMA_200"],
                mode='lines',
                name='200-day SMA',
                line=dict(color='#33CC33', width=1.5, dash='dot'),
                hovertemplate='<b>%{x|%Y-%m-%d}</b><br>SMA200: %{y:.2f}<extra></extra>'
            ))



            # Update layout
            fig.update_layout(
                title=f"{ticker} Historical Price",
                xaxis_title="Date",
                yaxis_title="Price",
                xaxis=dict(
                    type='date',
                    tickformat='%Y-%m-%d',
                    showgrid=True,
                    gridcolor='lightgrey',
                    rangeslider=dict(visible=False)
                ),
                yaxis=dict(
                    showgrid=True,
                    gridcolor='lightgrey'
                ),
                hovermode="x unified",
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                plot_bgcolor='white',
                margin=dict(l=50, r=50, b=50, t=80)
            )

            market_state = stock_data["Market_State"].iloc[-1]
            last_close = float(stock_data["Close"].iloc[-1])
            current_sma50 = float(stock_data["SMA_50"].iloc[-1])
            current_sma200 = float(stock_data["SMA_200"].iloc[-1])

            return [
                dcc.Graph(figure=fig),
                dbc.Row([
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                html.H5("Current Market Status", className="card-title"),
                                html.P(f"State: {market_state}", className="card-text"),
                                html.P(f"Close Price: {last_close:.2f}", className="card-text"),
                                html.P(f"50-day SMA: {current_sma50:.2f}", className="card-text"),
                                html.P(f"200-day SMA: {current_sma200:.2f}", className="card-text")
                            ])
                        ])
                    ], width=6),
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                html.H5("Market State Definition", className="card-title"),
                                html.P("Bull Market: When 50-day SMA is above 200-day SMA", className="card-text"),
                                html.P("Bear Market: When 50-day SMA is below 200-day SMA", className="card-text")
                            ])
                        ])
                    ], width=6)
                ], className="mt-3")
            ]

        # Monte Carlo Simulation Tab
        elif active_tab == "tab-montecarlo":
            # Monte Carlo Chart
            fig = go.Figure()
            for i in range(simulations.shape[1]):
                fig.add_trace(go.Scatter(
                    x=list(range(prediction_days + 1)),
                    y=simulations[:, i],
                    mode='lines',
                    line=dict(width=1, color='lightgrey'),
                    showlegend=False
                ))
            fig.add_trace(go.Scatter(
                x=list(range(prediction_days + 1)),
                y=np.mean(simulations, axis=1),
                mode='lines',
                name='Mean Price',
                line=dict(color='red', width=2, dash='dash')
            ))

            fig.update_layout(
                title=f"Monte Carlo Simulation for {ticker} ({prediction_days} days)",
                xaxis_title="Days Ahead",
                yaxis_title="Price ($)",
                hovermode="x unified",
                plot_bgcolor='white',
                xaxis=dict(showgrid=True, gridcolor='lightgray'),
                yaxis=dict(showgrid=True, gridcolor='lightgray')
            )

            # Diversification Advisor
            current_price = float(stock_data["Close"].iloc[-1])
            predicted_price = float(np.mean(final_prices))
            benchmark_volatility = get_benchmark_volatility()


            stock_value = 50000
            total_value = 80000
            weight_after = 0.8

            price_trend = get_price_trend(current_price, predicted_price)
            concentration_ratio, _ = check_concentration_ratio(stock_value, total_value)
            new_vol = calculate_portfolio_volatility(stock_volatility, benchmark_volatility, weight_after)
            advice_text = generate_advice(price_trend, concentration_ratio, stock_volatility, new_vol)

            advisor_card = dbc.Card([
                dbc.CardHeader("Diversification Advisor"),
                dbc.CardBody([
                    html.Pre(advice_text)
                ])
            ], className="mt-4")

            return [
                dcc.Graph(figure=fig),
                advisor_card
            ]

        # Price Distribution Tab
        elif active_tab == "tab-distribution":
            fig = go.Figure()
            fig.add_trace(go.Histogram(
                x=final_prices,
                nbinsx=50,
                marker_color='skyblue',
                opacity=0.7,
                name='Price Distribution'
            ))

            # Add vertical lines to show statistics results
            percentiles = {
                '5th': np.percentile(final_prices, 5),
                '15th': np.percentile(final_prices, 15),
                '85th': np.percentile(final_prices, 85),
                '95th': np.percentile(final_prices, 95),
                'Mean': np.mean(final_prices),
                'Median': np.median(final_prices)
            }

            colors = ['red', 'green', 'green', 'red', 'black', 'grey']
            line_styles = ['dash', 'dash', 'dash', 'dash', 'dash', 'dash']

            for i, (name, value) in enumerate(percentiles.items()):
                fig.add_vline(
                    x=value,
                    line=dict(color=colors[i], width=1, dash=line_styles[i]),
                    annotation_text=f"{name}: {value:.2f}",
                    annotation_position="top right"
                )

            fig.update_layout(
                title=f"Distribution of {ticker} Price in {prediction_days} Days",
                xaxis_title="Price ($)",
                yaxis_title="Frequency",
                bargap=0.1,
                plot_bgcolor='white',
                xaxis=dict(
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=0.5
                ),
                yaxis=dict(
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=0.5
                )
            )
            # Add table to show statistical results
            stats_table = dbc.Table([
                html.Thead(html.Tr([html.Th("Statistic"), html.Th("Value")])),
                html.Tbody([
                    html.Tr([html.Td("90% Confidence Interval"), html.Td(f"{future_price_90CI[0]} - {future_price_90CI[1]}")]),
                    html.Tr([html.Td("70% Confidence Interval"), html.Td(f"{future_price_70CI[0]} - {future_price_70CI[1]}")]),
                    html.Tr([html.Td("Mean Price"), html.Td(f"{mean_price:.2f}")]),
                    html.Tr([html.Td("Median Price"), html.Td(f"{median_price:.2f}")])
                ])
            ], bordered=True, className="mt-3")

            return [
                dcc.Graph(figure=fig),
                stats_table
            ]

        # Random Forest Tab
        elif active_tab == "tab-randomforest":
            days_list = list(range(1, prediction_days + 1))

            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=days_list,
                y=rf_predictions,
                mode='lines+markers',
                name='Predicted Price',
                line=dict(color='blue', width=2)
            ))
            fig.update_layout(
                title=f"Random Forest Prediction for {ticker} ({prediction_days} days)",
                xaxis_title="Days Ahead",
                yaxis_title="Price ($)",
                hovermode="x unified",
                plot_bgcolor='white',
                xaxis=dict(
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=0.5
                ),
                yaxis=dict(
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=0.5
                )
            )

            predictions_table = dbc.Table([
                html.Thead(html.Tr([html.Th("Day"), html.Th("Predicted Price")])),
                html.Tbody([
                    html.Tr([html.Td(f"Day {i+1}"), html.Td(f"{price:.2f}")])
                    for i, price in enumerate(rf_predictions)
                ])
            ], bordered=True, className="mt-3")

            return [
                dcc.Graph(figure=fig),
                predictions_table
            ]

        # Diversification Advisor Tab
        elif active_tab == "tab-advisor":
            # Advisor based on Monte Carlo
            mc_predicted_price = float(np.mean(simulations[-1]))
            mc_advice = run_diversification_advisor(current_price, mc_predicted_price, stock_volatility, benchmark_volatility, stock_value, total_portfolio_value, new_weight)


            # Advisor based on Random Forest
            rf_predicted_price = float(np.mean(rf_predictions))  # use average of predicted values
            rf_stock_volatility = float(stock_data["Log_Return"].std() * np.sqrt(252))  # historical std
            rf_advice = run_diversification_advisor(current_price, rf_predicted_price, rf_stock_volatility, benchmark_volatility, stock_value, total_portfolio_value, new_weight)


            return html.Div([
                dbc.Card([
                    dbc.CardHeader("Monte Carlo-Based Advisor"),
                    dbc.CardBody([html.Pre(mc_advice)])
                ], className="mb-4"),
                dbc.Card([
                    dbc.CardHeader("Random Forest-Based Advisor"),
                    dbc.CardBody([html.Pre(rf_advice)])
                ])
            ])


    except Exception as e:
        return dbc.Alert(f"Error: {str(e)}", color="danger")

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>